In [5]:
import tensorflow as tf
print("Version de TensorFlow :", tf.__version__)

import keras
print("Version de Keras :", keras.__version__)


Version de TensorFlow : 2.15.0
Version de Keras : 2.15.0


In [1]:
import tkinter as tk
from tkinter import filedialog
import threading
from keras.models import load_model
from PIL import Image
from PIL import ImageTk
import pickle
import tkinter as tk
from tkinter import filedialog
import threading
import os
# utilisée pour les calculs mathématiques en Python, spécialement pour les tableaux à plusieurs dimensions.
import numpy as np
#  convertir des données textuelles en séquences numériques qui peuvent être utilisées comme entrées pour les réseaux de neurones.
from keras.preprocessing.text import Tokenizer
# permet d'assurer que toutes les séquences de données ont la même longueur et peuvent donc être utilisées comme entrées pour le modèle,ajoutant des zéros à la fin des séquences plus courtes pour s'assurer que toutes les entrées ont la même longueur 
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Sequential est une classe de Keras qui permet de définir et créer des modèles de réseaux de neurones séquentiels en permettant d'ajouter les couches du modèle les unes après les autres de manière simple et conviviale
from keras.models import Sequential
# import des couches Embedding[vecteur de densite latente], GRU et Dense(effectuer les calculs de regression) pour le modèle RNN
from keras.layers import Embedding, GRU, Dense 
# stocker et manipuler des données tabulaires, telles que des tableaux de données enregistrés dans des fichiers CSV
import pandas as pd
import keras
import pickle 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from keras.models import load_model

In [2]:
from tensorflow.keras.models import load_model

# Charger le tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

# Charger le modèle sans réentraîner
loaded_model = load_model('sentiment_analysis_model.h5')


# Charger le modèle sans réentraîner
model = keras.models.load_model('happysadModel.h5')

In [7]:
from tkinter import Tk, Button, Toplevel, Label, filedialog, StringVar, CENTER, Entry
from PIL import Image, ImageTk
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def preprocess_image(image):
    
    image = image.resize((256, 256))
    image_array = np.array(image)
    image_array = image_array / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    return image_array


def predict(image):
    image_array = preprocess_image(image)
    prediction = model.predict(image_array)
    return prediction

class InputApplication(Tk):
    def __init__(self):
        super().__init__()
        self.title("")
        self.geometry("500x450") 

        self.input_type = StringVar()

        self.create_widgets()

    def create_widgets(self):
        self.bg_image = Image.open("baac.png")
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)
        self.bg_label = Label(self, image=self.bg_photo)
        self.bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)

        self.bind("<Configure>", self.on_resize)

        text_button = Button(self, text="Text", command=self.get_text_input, bg="#66c2ff", fg="white", padx=13, pady=8, font=('Helvetica', 12, 'bold'), relief="flat")
        text_button.place(relx=0.5, rely=0.4, anchor=CENTER, relwidth=0.6)

        image_button = Button(self, text="Image", command=self.select_image, bg="#66c2ff", fg="white", padx=13, pady=8, font=('Helvetica', 12, 'bold'), relief="flat")
        image_button.place(relx=0.5, rely=0.6, anchor=CENTER, relwidth=0.6)

    def on_resize(self, event):
        self.bg_image_resized = self.bg_image.resize((event.width, event.height))
        self.bg_photo_resized = ImageTk.PhotoImage(self.bg_image_resized)
        self.bg_label.configure(image=self.bg_photo_resized)
        self.bg_label.image = self.bg_photo_resized 

    def get_text_input(self):
        text_window = Toplevel(self)
        self.withdraw()  
        text_window.title("Text Input")
        text_window.geometry("500x450") 
        self.text_bg_photo = ImageTk.PhotoImage(self.bg_image_resized)
        self.text_bg_label = Label(text_window, image=self.text_bg_photo)
        self.text_bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)

        text_window.bind("<Configure>", self.on_resize_text_window)

        title_label = Label(text_window, text="Enter Text", font=('Helvetica', 18, 'bold'), bg=None, highlightthickness=0)
        title_label.place(relx=0.5, rely=0.1, anchor=CENTER)

        entry = Entry(text_window, font=('Helvetica', 24), bg=None, relief="flat", highlightthickness=0, bd=0)
        entry.place(relx=0.5, rely=0.4, anchor=CENTER, relwidth=0.6)
        entry.config(highlightbackground="#f5f5f5", highlightcolor="#f5f5f5")

        def submit_text():
            entered_text = entry.get()
            sequence = loaded_tokenizer.texts_to_sequences([entered_text])
            padded_sequence = pad_sequences(sequence, maxlen=150)
            prediction = loaded_model.predict(padded_sequence)[0][0]
            sentiment = "positive 😄" if prediction > 0.5 else "negative 😔"
            result_label.config(text=f"Your sentiment is: {sentiment}")

        submit_button = Button(text_window, text="Submit", command=submit_text, bg="#66c2ff", fg="white", padx=15, pady=8, font=('Helvetica', 12, 'bold'), relief="flat")
        submit_button.place(relx=0.5, rely=0.6, anchor=CENTER)

        result_label = Label(text_window, text="", font=('Helvetica', 23), bg=None, bd=0, highlightthickness=0)
        result_label.place(relx=0.5, rely=0.7, anchor=CENTER)

        def on_close():
            self.deiconify() 
            text_window.destroy()

        text_window.protocol("WM_DELETE_WINDOW", on_close)

    def on_resize_text_window(self, event):
        self.text_bg_photo_resized = ImageTk.PhotoImage(self.bg_image_resized.resize((event.width, event.height)))
        self.text_bg_label.configure(image=self.text_bg_photo_resized)
        self.text_bg_label.image = self.text_bg_photo_resized

    def select_image(self):
        image_path = filedialog.askopenfilename(title="Select Image", filetypes=(("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")))
        if image_path:
            image_window = Toplevel(self)
            self.withdraw()  
            image_window.title("Happy or Sad Image Classifier")
            image_window.geometry("400x400")

            image = Image.open(image_path)

            image = image.resize((300, 300))

            img = ImageTk.PhotoImage(image)
            label = Label(image_window, image=img, bg="#ffffff")
            label.image = img
            label.pack(pady=10)

            prediction = predict(image)
            sentiment = "Positive 😄" if prediction < 0.5 else "Negative 😔"

            sentiment_label = Label(image_window, text=f"Your sentiment is: {sentiment}", font=('Helvetica', 18, 'bold'), bg=None, bd=0, highlightthickness=0)
            sentiment_label.pack(pady=10)

            def on_close():
                self.deiconify()  
                image_window.destroy()

            image_window.protocol("WM_DELETE_WINDOW", on_close)

if __name__ == "__main__":
    app = InputApplication()
    app.mainloop()


1/1 [==============================] - 0s 109ms/step
